# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,52.83,75,100,8.01,NZ,1701285257
1,1,ust-nera,64.5667,143.2000,-14.58,99,74,5.91,RU,1701285111
2,2,nova sintra,14.8667,-24.7167,71.04,71,2,13.33,CV,1701285259
3,3,west island,-12.1568,96.8225,76.98,78,38,6.91,CC,1701285260
4,4,albany,42.6001,-73.9662,34.34,55,100,1.99,US,1701285237


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
vacation_plot = city_data_df.hvplot.points(
    x ='Lng',
    y= 'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.65,
    color = 'City')

# Display the map
vacation_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_city_df = city_data_df[(city_data_df['Max Temp']<=130) & (city_data_df['Max Temp']>35)
                        & (city_data_df['Humidity']<=80) & (city_data_df['Cloudiness']<=10)]
    
# Drop any rows with null values
perfect_city_df = perfect_city_df.dropna()
# Display sample data
perfect_city_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,nova sintra,14.8667,-24.7167,71.04,71,2,13.33,CV,1701285259
25,25,lebu,-37.6167,-73.6500,59.04,71,1,14.43,CL,1701285273
33,33,ad diwaniyah,31.9905,44.9306,57.83,42,0,10.71,IQ,1701285276
54,54,valparaiso,-33.0393,-71.6273,80.83,45,0,19.57,CL,1701285261
66,66,diawara,15.0220,-12.5437,87.53,13,0,7.43,SN,1701285290
67,67,hamilton,39.1834,-84.5333,46.65,52,0,14.97,US,1701285290
81,81,fort bragg,35.1390,-79.0060,49.91,28,0,8.05,US,1701285294
87,87,eten,-6.9081,-79.8642,77.25,61,0,19.57,PE,1701285295
91,91,rosarito,32.3333,-117.0333,67.12,60,4,6.71,MX,1701285296
93,93,port mathurin,-19.6833,63.4167,77.22,70,0,12.30,MU,1701285296


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#hotel_df = perfect_city_df[["City","Country","Lat","Lng","Humidity"]]
hotel_df = perfect_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,nova sintra,CV,14.8667,-24.7167,71,
25,lebu,CL,-37.6167,-73.6500,71,
33,ad diwaniyah,IQ,31.9905,44.9306,42,
54,valparaiso,CL,-33.0393,-71.6273,45,
66,diawara,SN,15.0220,-12.5437,13,
...,...,...,...,...,...,...
565,jiwani,PK,25.0500,61.7417,65,
571,spearfish,US,44.4908,-103.8594,13,
576,carahue,CL,-38.7000,-73.1667,49,
577,sarpang,BT,26.8640,90.2674,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "apiKey":geoapify_key,
        "format":"json",
        "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"] 
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
nova sintra - nearest hotel: Residência Ka Dencho
lebu - nearest hotel: No hotel found
ad diwaniyah - nearest hotel: فندق ومطعم الديوانية
valparaiso - nearest hotel: Ibis Hotel
diawara - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
fort bragg - nearest hotel: Airborne Inn Lodging
eten - nearest hotel: No hotel found
rosarito - nearest hotel: WorldMark La Paloma
port mathurin - nearest hotel: Escale Vacances
diu - nearest hotel: Hotel Ankur
taoudenni - nearest hotel: No hotel found
fort dodge - nearest hotel: Days Inn by Wyndham Fort Dodge
gaiman - nearest hotel: yr hen ffordd
port alfred - nearest hotel: No hotel found
mbeni - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
plettenberg bay - nearest hotel: Milkwood Manor
kandiari - nearest hotel: No hotel found
demopolis - nearest hotel: No hotel found
constitucion - nearest hotel: Hostal Alameda
south venice - nearest hotel: Kon-Tiki Motel
carnarvon - nearest hotel: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,nova sintra,CV,14.8667,-24.7167,71,Residência Ka Dencho
25,lebu,CL,-37.6167,-73.6500,71,No hotel found
33,ad diwaniyah,IQ,31.9905,44.9306,42,فندق ومطعم الديوانية
54,valparaiso,CL,-33.0393,-71.6273,45,Ibis Hotel
66,diawara,SN,15.0220,-12.5437,13,No hotel found
...,...,...,...,...,...,...
565,jiwani,PK,25.0500,61.7417,65,No hotel found
571,spearfish,US,44.4908,-103.8594,13,Best Western Black Hills Lodge
576,carahue,CL,-38.7000,-73.1667,49,No hotel found
577,sarpang,BT,26.8640,90.2674,64,No hotel found


In [12]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x ='Lng',
    y= 'Lat',
    geo = True,
    hover_cols = ['City', 'Country'],
    tiles = 'OSM',
    size = 'Humidity',
    scale = 0.75,
    color = 'Hotel Name')

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)